In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import caselawnet
import json
from caselawnet import echrapi as echr

[nltk_data] Downloading package punkt to /home/carlosm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def buildEdge(source, target):
    id = source + '_' + target
    return {
      "source": source,
      "target": target,
      "id": id
    }

In [4]:
kpthesaurus_keyword = '(Art. 1) Jurisdiction of States'
nodes = echr.getDataForKeyword(kpthesaurus_keyword)

print('Initial results: ', len(nodes))

Initial results:  80


For each case mentioning article 1
 - Find all cases which cite these cases
 

In [5]:
citations = {}
for node in nodes:
    appno = node['appno']
    if appno in citations:
        print('AppNo %s already included -- skip'%appno)
    else:
        # print('calling for ',appno)
        cases = echr.getCasesCiting(appno)
        citations[appno] = cases


AppNo 48787/99 already included -- skip
AppNo 20652/92 already included -- skip
AppNo 6231/73 already included -- skip
AppNo 45036/98 already included -- skip
AppNo 15318/89 already included -- skip


Index cases by appno

In [6]:
nodeDict = {}
for node in nodes:
    if node['appno'] not in nodeDict:
        nodeDict[node['appno']] = node
    else:
        print('appno already known -- skip')

for appno, refNodes in citations.items():
    for node in refNodes:
        if node['appno'] not in nodeDict:
            nodeDict[node['appno']] = node


appno already known -- skip
appno already known -- skip
appno already known -- skip
appno already known -- skip
appno already known -- skip


In [7]:
nodesAll = nodeDict.values()
nodesAll = list(nodesAll) # 80 base cases + all cases that cite them
print('Total number of cases considered: ', len(nodesAll))

Total number of cases considered:  697


In [8]:
#For each node in nodesAll
# - Get its appNo and refersTo
# - build table of links (appno, refertTo*)

#For each link in table of links
# - Get the link source and target appNos
# - Check source and target appNos exist in nodeDict
#   - If they do, preserve link


links = []

for node in nodesAll:
    appno = node['appno']
    refers = node['refersTo']
    for refAppNo in refers:
        links.append((appno, refAppNo))
links = list(set(links))

print('In total, out cases contain a total number of citations: ', len(links))


In total, out cases contain a total number of citations:  18160


In [9]:
appnos = [ node['appno'] for node in nodeDict.values() ]

filtered_links = []
for source_appno, target_appno in links:
    if (source_appno in appnos) and (target_appno in appnos):
        source_id = nodeDict[source_appno]['id']
        target_id = nodeDict[target_appno]['id']
        filtered_links.append((source_id, target_id))

print('But we are only interested in the citations inside our set of cases: ', len(filtered_links))

But we are only interested in the citations inside our set of cases:  3234


In [10]:
edges = [ buildEdge(source, target) for source, target in list(set(filtered_links)) ]
nodesAllEnriched = caselawnet.network_analysis.add_network_statistics(nodesAll, edges)

network = {
    'title': kpthesaurus_keyword,
    'nodes': nodesAllEnriched,
    'edges': edges
}

with open('juristiction.json', 'w') as fout:
    net_str = json.dumps(network)
    fout.write(net_str)